In [1]:
from cv2 import namedWindow, VideoCapture, cvtColor, COLOR_BGR2RGB, addWeighted, imshow, waitKey, destroyAllWindows
from PIL import Image
from numpy import argmax, asarray, load
import cv2
from tensorflow.keras.models import load_model
from os import path

In [2]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
!pip install tensorflow

In [ ]:
!pip install pillow

In [ ]:
# получение пути исполняемого файла
detector2_path=path.abspath(open('detector.ipynb').name)

# получение директории исполняемого файла
main_dir_path=path.dirname(path.abspath(detector2_path))

# получение пути emoji_array.npy
emoji_array_path=path.join(main_dir_path, 'emoji_array.npy')

# получение пути model.h5
model_path=path.join(main_dir_path, 'model_old.h5')

In [ ]:
# получение массива emoji модели
emoji_array=load(emoji_array_path)
model=load_model(model_path)

In [ ]:
# создание окна трансляции с камеры
namedWindow('Emotion Detector')

# получение первой в списке камеры
cap=VideoCapture(0)

while True:
    # считывание кадров (frame) с камеры
    ret,frame=cap.read()
    
    # преобразование кадров в однокональные изображения
    frame_2=frame[0:len(frame[1]),0:len(frame[0]),1]
    
    # преобразование кадров в формат для PIL
    img=Image.fromarray(frame_2)
    
    # преобразование кадров в равносторонние изображения размером 48 на 48
    if img.size[0]>64 or img.size[1]>64:
        if img.size[0]>img.size[1]:
            left=(img.size[0]-img.size[1])/2
            right=img.size[0]-left
        else:
            left=(img.size[1]-img.size[0])/2
            right=img.size[1]-left
            
    img=img.crop((left,0,right,img.size[1])).resize((64,64))
        
    # добавление размерности для model.predict()
    img=asarray(img)
    img=img[None,...]
    
    # предстказание
    img=emoji_array[argmax(model.predict(img))]
    
    # преобразование предстказания в трехканальное изображение
    img=cvtColor(img,COLOR_BGR2RGB)
    
    # наложение изоображения предстказания на трансляцию камеры
    result=addWeighted(frame[320:480,480:640],1,img,0.5,0)
    frame[320:480, 480:640]=result
    
    # отрисовка frame со всеми вышеуказанными преобразованиями в реальном времени в окне namedWindow('Emotion Detector')
    imshow('Emotion Detector',frame)
    
    # остановка программы по нажатию escape
    if waitKey(1)==27:
        break

# освобождение камеры
cap.release()

# закрытие окна
destroyAllWindows()